#Trần Hoàn Đức Duy - 19521434

#Bài tập 1: sử dụng GBM để thực hiện trên dữ liệu sau

In [1]:
import pandas as pd

In [2]:
ds = pd.read_csv('/content/drive/MyDrive/Programming/Python for machine learning/data/Data.csv')
ds.tail()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
678,776715,3,1,1,1,3,2,1,1,1,2
679,841769,2,1,1,1,2,1,1,1,1,2
680,888820,5,10,10,3,7,3,8,10,2,4
681,897471,4,8,6,4,3,4,10,6,1,4
682,897471,4,8,8,5,4,5,10,4,1,4


In [3]:
X,y = ds.iloc[:,:-1].to_numpy(),ds.iloc[:,-1].to_numpy()

In [4]:
print('Classes:',*set(y))

Classes: 2 4


In [5]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np

def evaluation_kfold(model,X,y):
  kf = KFold(n_splits=5)
  train_accuracies = []
  val_accuracies = []
  for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    scaler = StandardScaler()
    scaler = scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    
    model.fit(X_train,y_train)

    train_accuracies.append(model.score(X_train,y_train))
    val_accuracies.append(model.score(X_val,y_val))
  
  print('Train accuracy: ',np.mean(train_accuracies))
  print('Validation accuracy: ',np.mean(val_accuracies))
  return model,scaler

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

In [7]:
DB = GradientBoostingClassifier()

_,_=evaluation_kfold(DB,X,y)

Train accuracy:  1.0
Validation accuracy:  0.9605195362816661


#Bài tập 2: sử dụng XGBoost để thực hiện trên dữ liệu của bài tập 1

In [8]:
import xgboost

In [9]:
XGB = xgboost.XGBClassifier()

_,_ = evaluation_kfold(XGB,X,y)

Train accuracy:  0.9959733745839779
Validation accuracy:  0.9575998282524688


#Bài tập 3: làm bài tập 1 và bài tập 2 với dataset sau

##Data preprocessing

In [10]:
ds = pd.read_csv('/content/drive/MyDrive/Programming/Python for machine learning/data/loan_data.csv',na_values='?')
ds.tail()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N


In [11]:
ds.Dependents = ds.Dependents.apply(lambda x: str(3) if x == '3+' else str(x))
ds.Loan_Status = ds.Loan_Status.apply(lambda x: 0 if x=='N' else 1)
ds.tail()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,1
610,LP002979,Male,Yes,3,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,1
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,1
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,1
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,0


In [12]:
ds.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents            0
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [13]:
ds = ds.dropna()
ds = ds.iloc[:,1:]
ds

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,1
610,Male,Yes,3,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,1
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,1
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,1


In [14]:
X,y = ds.iloc[:,:-1].to_numpy(), ds.iloc[:,-1].to_numpy()

In [15]:
from sklearn.preprocessing import OneHotEncoder, Normalizer,LabelEncoder
from sklearn.compose import ColumnTransformer
def evaluation_kfold(model,X,y):
  kf = KFold(n_splits=5)
  train_accuracies = []
  val_accuracies = []
  for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    columnTransfer_x = ColumnTransformer([('encoder', OneHotEncoder(sparse=False),[0,1,2,3,4,9,10]), ('norm', Normalizer(), [5,6,7,8])], remainder='passthrough')
    scaler = columnTransfer_x.fit(X_train)

    X_train = columnTransfer_x.transform(X_train)
    X_val = columnTransfer_x.transform(X_val)

    model.fit(X_train,y_train)
    train_accuracies.append(model.score(X_train,y_train))
    val_accuracies.append(model.score(X_val,y_val))

  print('Train accuracy: ',np.mean(train_accuracies))
  print('Validation accuracy: ',np.mean(val_accuracies))

  return model

##GBM

In [16]:
GB = GradientBoostingClassifier()

_ = evaluation_kfold(GB,X,y)

Train accuracy:  0.9341836734693878
Validation accuracy:  0.789795918367347


##XGBoost

In [17]:
XGB = xgboost.XGBClassifier()

_ = evaluation_kfold(XGB,X,y)

Train accuracy:  0.8913265306122448
Validation accuracy:  0.7979591836734693
